In [1]:
import os
import argparse
import datetime

import torch
from torch import nn
import numpy as np
import gymnasium as gym

from tianshou.data import Collector, Batch, to_torch
from tianshou.data.types import RolloutBatchProtocol
from tianshou.data.buffer.vecbuf import VectorReplayBuffer, ReplayBuffer
from tianshou.env import SubprocVectorEnv
from tianshou.policy import SACPolicy, BasePolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import ActorProb, Critic
from examples.offline.utils import load_buffer_d4rl
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
from tianshou.trainer import OffpolicyTrainer

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 303, in register_gymnasium_envs
    _register_dm_control_envs()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 63, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shim

In [2]:
device = "cuda:3"

In [3]:
def get_sac_args():
    args = argparse.Namespace(
        task="Hopper-v2",
        buffer_size=1000000,
        hidden_sizes=[256, 256, 256],
        actor_lr=3e-4,
        critic_lr=3e-4,
        gamma=0.99,
        tau=0.005,
        alpha=0.1,
        start_timesteps=1,
        epoch=200,
        step_per_epoch=5000,
        step_per_collect=1,
        update_per_step=1,
        batch_size=256,
        training_num=1,
        test_num=10,
        device=device,
        norm_layer=True,
    )
    return args

In [4]:
args = get_sac_args()
env = gym.make(args.task)

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [5]:
def load_policy(path):
    args = get_sac_args()
    env = gym.make(args.task)
    args.state_shape = env.observation_space.shape or env.observation_space.n
    args.action_shape = env.action_space.shape or env.action_space.n
    args.max_action = env.action_space.high[0]
    # model
    net_a = Net(args.state_shape, hidden_sizes=args.hidden_sizes, device=args.device)
    actor = ActorProb(
        net_a,
        args.action_shape,
        device=args.device,
        unbounded=True,
        conditioned_sigma=True,
    ).to(args.device)
    actor_optim = torch.optim.Adam(actor.parameters(), lr=args.actor_lr)
    net_c1 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=args.device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    net_c2 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=args.device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    critic1 = Critic(net_c1, device=args.device).to(args.device)
    critic1_optim = torch.optim.Adam(critic1.parameters(), lr=args.critic_lr)
    critic2 = Critic(net_c2, device=args.device).to(args.device)
    critic2_optim = torch.optim.Adam(critic2.parameters(), lr=args.critic_lr)
    policy = SACPolicy(
        actor,
        actor_optim,
        critic1,
        critic1_optim,
        critic2,
        critic2_optim,
        tau=args.tau,
        gamma=args.gamma,
        alpha=args.alpha,
        action_space=env.action_space,
    )
    policy.load_state_dict(torch.load(path, map_location=args.device))
    print("Loaded agent from: ", path)
    return policy

In [6]:
cql_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/cql/0/231220-111219/policy.pth")

Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/cql/0/231220-111219/policy.pth


In [7]:
sac_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/sac/0/231219-163624/policy.pth")

Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/sac/0/231219-163624/policy.pth


/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [8]:
envs = SubprocVectorEnv([lambda: gym.make(args.task) for _ in range(10)])

In [11]:
cql_policy.eval()
cql_collector = Collector(cql_policy, envs)
cql_result = cql_collector.collect(n_episode=40)
cql_result

{'n/ep': 40,
 'n/st': 29361,
 'rews': array([1489.02481167, 1842.21185885, 1851.54305439, 1895.97065876,
        2116.01603433, 2210.91089589, 2403.31832445, 2503.1987029 ,
        3191.59420128, 3298.24723624, 1874.35273658, 1882.85695515,
        1927.46393929, 3121.14735053, 2855.48096741, 2810.40096311,
        2802.10252388, 2675.67930741, 1928.14911438, 2158.49601966,
        2433.0786285 , 3336.26060991, 2102.4805384 , 2180.70791761,
        1875.98584251, 3289.97193547, 2985.8541246 , 3122.38828219,
        3333.63136507, 2438.59254587, 1625.10616891, 3222.07709866,
        3020.51567111, 2684.40105251, 2162.56228329, 2420.0371619 ,
        1604.1628094 , 1564.49197052, 3314.09554543, 2200.40869673]),
 'lens': array([ 457,  559,  562,  575,  638,  669,  717,  760,  946, 1000,  566,
         573,  584,  926,  856,  836,  842,  799,  583,  653,  727,  996,
         629,  652,  567, 1000,  894,  936, 1000,  734,  496,  958,  893,
         798,  651,  724,  484,  475,  980,  666]),

In [9]:
sac_policy.eval()
sac_collector = Collector(sac_policy, envs)
sac_result = sac_collector.collect(n_episode=40)
sac_result

{'n/ep': 40,
 'n/st': 38784,
 'rews': array([1374.48870402, 3563.14717783, 3557.86708206, 3608.04713137,
        3548.05139858, 3553.03855107, 3536.79429071, 3562.16889125,
        3618.95147274, 3517.54973909, 3552.13703624, 3566.05168652,
        3574.5577577 , 3612.51821892, 3572.29116713, 3547.21679116,
        3545.03262353, 3689.45713996, 3574.47625481, 3566.61674521,
        3599.40311662, 3639.44434494, 3526.78525297, 3546.06227241,
        3518.65236275, 3586.98774992, 3611.27413258, 3619.59866963,
        3563.55801092, 3556.8661537 , 1370.73038917, 3551.69977827,
        3572.18721047, 3557.7686918 , 3541.53283945, 3593.61391383,
        3567.80575847, 3569.23964549, 3636.30429332, 3574.10492874]),
 'lens': array([ 393, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  391, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000]),

In [81]:
from tianshou.data import ReplayBuffer
from tianshou.data.types import RolloutBatchProtocol


class MixedPolicy(BasePolicy):
    def __init__(self, base_policy: BasePolicy, expert_policy: BasePolicy, action_space, buffer):
        super().__init__(action_space=action_space, action_scaling=True)
        self.base_policy = base_policy
        self.expert_policy = expert_policy
        self.device = device
        self.buffer = buffer

    def forward(self, batch: RolloutBatchProtocol, state=None, **kwargs):
        bsz = len(batch.obs)
        batch = to_torch(batch, dtype=torch.float32, device=self.device)
        self.base_policy.eval()
        self.expert_policy.eval()
        with torch.no_grad():
            expert_result = self.expert_policy(batch)
            base_result = self.base_policy(batch)
            expert_qvalues1 = self.expert_policy.critic1(batch.obs, expert_result.act)
            expert_qvalues2 = self.expert_policy.critic2(batch.obs, expert_result.act)
            expert_qvalues = torch.minimum(expert_qvalues1, expert_qvalues2)
            base_qvalues1 = self.base_policy.critic1(batch.obs, base_result.act)
            base_qvalues2 = self.base_policy.critic2(batch.obs, base_result.act)
            base_qvalues = torch.minimum(base_qvalues1, base_qvalues2)
        cede_ctrl = base_qvalues < expert_qvalues
        actions = torch.where(cede_ctrl, expert_result.act, base_result.act)
        return Batch(**{'act': actions, 'policy': Batch({'cede_ctrl': cede_ctrl})})

    def train(self, mode: bool = True) -> "MixedPolicy":
        self.base_policy.train(mode)
        return self
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.base_policy.process_fn(batch, buffer, indices)

    def learn(self, batch, **kwargs):
        # cede_ctrl = batch.policy.cede_ctrl.cpu().squeeze()
        # train_batch = batch[~cede_ctrl]
        # info = self.base_policy.learn(train_batch)
        info = self.base_policy.learn(batch)
        return info

In [12]:
test_buffer = VectorReplayBuffer(5000, 5)

In [82]:
mixed_policy = MixedPolicy(cql_policy, sac_policy, env.action_space, test_buffer)

In [12]:
mixed_collector = Collector(mixed_policy, envs)
mixed_result = mixed_collector.collect(n_episode=40)
mixed_result

{'n/ep': 40,
 'n/st': 39442,
 'rews': array([2877.85420345, 3579.38976417, 3566.2887685 , 3612.26257751,
        3648.73333248, 3655.45149677, 3559.89113674, 3552.6419363 ,
        3588.0665408 , 3578.10143437, 3576.98877615, 3561.46943533,
        3558.70658515, 3598.58161738, 3558.16991895, 3525.51417879,
        3579.98865515, 3558.74737203, 3518.58784384, 3641.33416581,
        3563.76800552, 3231.92112228, 3558.71492095, 3535.73037386,
        3558.96929922, 3583.61728139, 3556.20513785, 3542.30987626,
        3584.5683532 , 3573.77252436, 2903.35971768, 3574.50541953,
        3551.28549583, 3540.74278544, 3513.75433021, 3546.94601085,
        3569.82593565, 3520.37927402, 3546.47417846, 3562.03266267]),
 'lens': array([ 798, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  869,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  775, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000]),

In [13]:
class DeterministicPolicy(BasePolicy):
    def __init__(self, policy, action_space):
        super().__init__(action_space=action_space, action_scaling=True)
        self.policy = policy

    def train(self, mode: bool = True) -> "DeterministicPolicy":
        self.policy.train(mode)
        return self
    
    def forward(self, batch, state=None, **kwargs):
        self.policy.eval()
        return self.policy(batch)
    
    def learn(self, batch, **kwargs):
        info = self.policy.learn(batch)
        return info
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.policy.process_fn(batch, buffer, indices)

In [14]:
det_cql = DeterministicPolicy(cql_policy, env.action_space)

In [9]:
test_envs = SubprocVectorEnv([lambda: gym.make(args.task) for _ in range(5)])

In [10]:
offline_data = load_buffer_d4rl("hopper-medium-v2")

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile:   0%|          | 0/21 [00:00<?, ?it/s]

load datafile: 100%|██████████| 21/21 [00:01<00:00, 17.56it/s]


In [12]:
policy = cql_policy

In [13]:
test_buffer = VectorReplayBuffer(5000, 5)
train_collector = Collector(policy, env, offline_data)
test_collector = Collector(policy, test_envs, test_buffer)

/data/user/R901105/dev/my_fork/tianshou/tianshou/data/collector.py:70: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [61]:
test_collector.collect(n_episode=1)

{'n/ep': 1,
 'n/st': 1000,
 'rews': array([3536.64228072]),
 'lens': array([1000]),
 'idxs': array([0]),
 'rew': 3536.6422807150416,
 'len': 1000.0,
 'rew_std': 0.0,
 'len_std': 0.0}

In [14]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = os.path.join(args.task, policy.__class__.__name__, now)
log_path = os.path.join("/data/user/R901105/dev/log", log_name)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)
print(log_path)

/data/user/R901105/dev/log/Hopper-v2/SACPolicy/240122-103110


In [15]:
def get_returns(policy, return_cede_ctrl=False):
    policy.eval()
    returns = []
    cede_ctrl = []
    for _ in range(5):
        obs, info = env.reset()
        done = False
        cum_reward = 0
        actions = []
        if return_cede_ctrl:
            cctrl_ep = []
        while not done:
            batch = Batch(obs=torch.from_numpy(np.expand_dims(obs, 0)).to(device), info=info)
            with torch.no_grad():
                result = policy(batch)
            act = result.act.cpu().squeeze().numpy()
            act = policy.map_action(act)
            if return_cede_ctrl:
                cctrl_ep.append(result.policy.cede_ctrl.cpu().squeeze().numpy())
            obs, reward, terminated, truncated, info = env.step(act)
            actions.append(act)
            cum_reward += reward
            done = terminated or truncated
        returns.append(cum_reward)
        if return_cede_ctrl:
            cede_ctrl.append(np.array(cctrl_ep).mean())
    if return_cede_ctrl:
        return np.array(returns),  np.array(cede_ctrl).mean()
    return np.array(returns), _

In [16]:
def test_fn(num_epoch: int, step_idx: int):
    returns, cede_ctrl = get_returns(policy, True)
    print(returns.mean(), cede_ctrl.mean())

In [17]:
result = OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    # test_fn=test_fn,
    max_epoch=200,
    step_per_epoch=1000,
    step_per_collect=1,
    episode_per_test=1,
    batch_size=512,
    logger=logger,
    update_per_step=1,
    test_in_train=False,
).run()

Epoch #1:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch #1: 1001it [00:16, 60.77it/s, env_step=1000, gradient_step=1000, len=98, loss/actor=-268.244, loss/critic1=3.523, loss/critic2=3.553, n/ep=0, n/st=1, rew=229.69]                          


Epoch #1: test_reward: 3272.901825 ± 0.000000, best_reward: 3272.901825 ± 0.000000 in #1


Epoch #2: 1001it [00:14, 67.96it/s, env_step=2000, gradient_step=2000, len=1000, loss/actor=-267.586, loss/critic1=3.446, loss/critic2=3.416, n/ep=0, n/st=1, rew=3283.77]                          


Epoch #2: test_reward: 3240.190149 ± 0.000000, best_reward: 3272.901825 ± 0.000000 in #1


...

Epoch #198: test_reward: 3326.646583 ± 0.000000, best_reward: 3434.357724 ± 0.000000 in #188


Epoch #199: 1001it [00:14, 68.35it/s, env_step=199000, gradient_step=199000, len=254, loss/actor=-290.538, loss/critic1=16.891, loss/critic2=15.512, n/ep=0, n/st=1, rew=891.34]                          


Epoch #199: test_reward: 3390.424575 ± 0.000000, best_reward: 3434.357724 ± 0.000000 in #188


Epoch #200: 1001it [00:14, 68.58it/s, env_step=200000, gradient_step=200000, len=1000, loss/actor=-290.436, loss/critic1=16.887, loss/critic2=18.142, n/ep=0, n/st=1, rew=3396.41]                          


Epoch #200: test_reward: 3372.969074 ± 0.000000, best_reward: 3434.357724 ± 0.000000 in #188
